In [ ]:
import sys
sys.path.append("../cv-tfq")

!pip install cirq
!pip install tensorflow
!pip install tensorflow_quantum

import cirq
import sympy
import numpy as np

import tensorflow as tf
import tensorflow_quantum as tfq

from cv_ops import BinaryOp, PositionOp, MomentumOp
from cv_subroutines import ComputationalLayerBinary, ComputationalLayerInteger, QFT, centeredQFT, kick_position, kick_momentum, discrete_continuous

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 511.7 MB 19.5 MB/s eta 0:00:011     |█████████████████████           | 335.7 MB 190.7 MB/s eta 0:00:01     |███████████████████████████     | 433.0 MB 7.1 MB/s eta 0:00:12     |█████████████████████████████▉  | 476.6 MB 3.8 MB/s eta 0:00:10     |██████████████████████████████▍ | 485.2 MB 3.8 MB/s eta 0:00:07
     |████████████████████████████████| 1.6 MB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 28.9 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 10.4 MB/s ta 0:00:01
     |████████████████████████████████| 123 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 438 kB 19.6 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 10.7 MB/s eta 0:00:01
     |██████████████████████

     |████████████████████████████████| 151 kB 9.1 MB/s eta 0:00:01
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=f9dad373308594e8a4a78c58591f9db9e81320ef73a7c014f1f9d307d798a6ae
  Stored in directory: /tmp/pip-ephem-wheel-cache-6q8btmav/wheels/b6/0d/90/0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
Successfully built termcolor
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 10.5 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 100 kB 17.5 MB/s ta 0:00:01
     |████████████████████████████████| 90 kB 14.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 15.6 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 14.5 MB/s eta 0:00:01
   

     |████████████████████████████████| 91 kB 94.8 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 14.0 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 18.4 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 18.4 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 15.9 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 25.6 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 8.3 MB/s eta 0:00:011
     |████████████████████████████████| 45 kB 15.6 MB/s eta 0:00:01
INFO: pip is looking at multiple versions of pillow to determine which version is compatible with other requirements. 

     |████████████████████████████████| 40 kB 7.9 MB/s eta 0:00:01
INFO: pip is looking at multiple versions of packaging to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. If you want to abort this run, you can press Ctrl + C to do so. To improve how pip performs, tell us what happened here: https://pip.pypa.io/surveys/backtracking
INFO: pip is looking at multiple versions of numpy to determine which version is compatible with other requirements. This could take a while.
     |████████████████████████████████| 17.1 MB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 17.1 MB 12.9 MB/s eta 0:00:01     |████████████████████████▉       | 13.2 MB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 16.8 MB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 16.8 MB 13.8 MB/s eta 0:00:01

     |████████████████████████████████| 6.5 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.5 MB 7.2 MB/s eta 0:00:01


In [ ]:
def U_mix(qubits, a, b, gamma):
    d = 2 ** len(qubits)
    return centeredQFT(qubits, inverse=True) + tfq.util.exponential(operators = [((1/2) * MomentumOp(qubits).op ** 2], coefficients = gamma) + centeredQFT(qubits)

def H_cost_poly(qubits, poly, a, b):
    H_c = cirq.Circuit()
    for q in qubits:
        H_c += cirq.PauliString(poly[0]*cirq.I(q))
    for idx, p in enumerate(poly[1:]):
        H_c = p * PositionOp(qubits).op
        for _ in range(idx):
            H_c *= PositionOp(qubits).op

    return H_c

In [ ]:
def generate_parameters(p):
    param_list = ['a','b']
    for _ in range(p - 1):
        param_list += [chr(ord(param_list[-1]) + 1), chr(ord(param_list[-1]) + 2)]
    param_string = param_list[0]
    for param in param_list[1:]:
        param_string += ' ' + param
    return sympy.symbols(param_string)

def qaoa_circuit(qubits, poly, a, b, p, qaoa_parameters):
    circuit = cirq.Circuit()
    for idx in range(p):
        circuit += tfq.util.exponential(operators = [H_cost_poly(qubits, poly, a, b)], coefficients = qaoa_parameters[2*idx:2*idx+1])
        circuit += U_mix(qubits, a, b, qaoa_parameters[2*idx+1:2*idx+2])
    return circuit

def hadamard_circuit(qubits):
    return cirq.Circuit([cirq.H(q) for q in qubits])

In [ ]:
def run_qaoa():
    poly_to_optimize = [10, -6, 1]
    a, b = -10, 10
    qubits = [cirq.GridQubit(0,i) for i in range(10)]
    p = 2
    qaoa_parameters = generate_parameters(p)
    
    model_circuit = qaoa_circuit(qubits, poly_to_optimize, a, b, p, qaoa_parameters)
    model_readout = H_cost_poly(qubits, poly_to_optimize, a, b)
    input_ = [hadamard_circuit(qubits)]
    input_ = tfq.convert_to_tensor(input_)
    optimum = np.array([0])

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(), dtype=tf.dtypes.string))
    model.add(tfq.layers.PQC(model_circuit, model_readout))
    tfq.layers.PQC

    model.compile(loss=tf.keras.losses.mean_absolute_error,
                  optimizer=tf.keras.optimizers.Adam())

    history = model.fit(input_, optimum, epochs=1000, verbose=1)